## Convert behavior data

In [23]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_context('paper')
sns.set_style('ticks')

In [24]:
import numpy as np
import pandas as pd
from glob import glob
from os.path import join, exists
from shutil import copytree
import json

###Set directory and session information

In [25]:
directory = '/tier2/freeman/Nick/lfov.calibration'

In [26]:
key = '2016-05-15'
name = 'anm-05001-L23'

Check if behavior data exists

In [34]:
path = join(directory, 'unprocessed', 'behavior', key, name)
print exists(path)
name = '5001-L23'

False


###Load trial data

In [28]:
files = glob(join(path, 'trial-*.csv'))
files.sort()
trials = [pd.read_csv(x) for x in files]

Create session table

In [29]:
session = pd.concat(trials)
session.reset_index(inplace=True)
session.rename(columns={'index': 'tick'}, inplace=True)

Add global time

In [30]:
trial = session.groupby(['number']).last()
offsets = np.concatenate(([0],trial.time.values))
offsets = np.cumsum(offsets)
session['allTime'] = session.time + offsets[session.number]

Add filtered speed

In [31]:
speed = (session.xSpeed.pow(2)+session.ySpeed.pow(2)).pow(.5)
session['speed'] = np.convolve(speed,np.ones(50)/50, mode='same')    

### Save full-length session

In [32]:
session.to_csv(join(path, 'session.csv'))

###Copy data to new directory

In [33]:
savepath = join(directory, 'reprocessed', name, key, 'behavior')
copytree(path,savepath)